In [ ]:
!pip install spacy torch
!python -m spacy download ru_core_news_sm
!pip install docx2txt
# Установка djvulibre
!apt-get update
!apt-get install -y djvulibre-bin

!pip install langdetect
!pip install beautifulsoup4


!pip install pymupdf
!pip install python-docx
import subprocess
import fitz  # PyMuPDF
import spacy
from langdetect import detect
from bs4 import BeautifulSoup
import requests
from docx import Document

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:

# Функция для извлечения текста из PDF
def extract_text_from_pdf(file_path):
    try:
        document = fitz.open(file_path)
        text = ""
        for page_num in range(len(document)):
            page = document.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Ошибка при чтении PDF: {e}")
        return ""

# Функция для извлечения текста из DOCX
def extract_text_from_docx(file_path):
    try:
        doc = Document(file_path)
        full_text = []
        for para in doc.paragraphs:
            full_text.append(para.text)
        return '\n'.join(full_text)
    except Exception as e:
        print(f"Ошибка при чтении DOCX: {e}")
        return ""

# Функция для извлечения текста из DOC
def extract_text_from_doc(file_path):
    try:
        result = subprocess.run(['antiword', file_path], stdout=subprocess.PIPE, check=True)
        return result.stdout.decode('utf-8')
    except Exception as e:
        print(f"Ошибка при чтении DOC: {e}")
        return ""

# Функция для извлечения текста из DJVU
def extract_text_from_djvu(file_path):
    try:
        result = subprocess.run(['djvutxt', file_path, 'output.txt'], stdout=subprocess.PIPE, check=True)
        with open('output.txt', 'r') as file:
            text = file.read()
        return text
    except Exception as e:
        print(f"Ошибка при чтении DJVU: {e}")
        return ""

# Функция для синтаксического анализа текста
def analyze_text(text):
    try:
        # Удаляем лишние пробелы и символы новой строки
        text = ' '.join(text.split())

        language = detect(text)
        if language == 'en':
            nlp = spacy.load("en_core_web_sm")
        elif language == 'ru':
            nlp = spacy.load("ru_core_news_sm")
        else:
            print(f"Модель для языка {language} не найдена. Используется модель по умолчанию (английский).")
            nlp = spacy.load("en_core_web_sm")  # Используем английскую модель по умолчанию

        doc = nlp(text)
        for token in doc:
            print(f"{token.text} ({token.lemma_}) --> {token.pos_} [{token.dep_}]")
    except Exception as e:
        print(f"Ошибка при анализе текста: {e}")

# Функция для парсинга HTML
def parse_html(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            # Попробуем определить кодировку
            encoding = response.apparent_encoding
            html_content = response.content.decode(encoding)
            soup = BeautifulSoup(html_content, 'html.parser')
            paragraphs = soup.find_all('p')
            full_text = ' '.join(paragraph.text.strip() for paragraph in paragraphs)
            print("Извлеченный текст из HTML:")
            print(full_text)
            analyze_text(full_text)
        else:
            print(f"Ошибка при загрузке страницы: {response.status_code}")
    except Exception as e:
        print(f"Ошибка при парсинге HTML: {e}")

# Запрос путей к файлам и URL у пользователя
pdf_path = input("Введите путь к PDF файлу (или нажмите Enter, чтобы пропустить): ")
docx_path = input("Введите путь к DOCX файлу (или нажмите Enter, чтобы пропустить): ")
doc_path = input("Введите путь к DOC файлу (или нажмите Enter, чтобы пропустить): ")
djvu_path = input("Введите путь к DJVU файлу (или нажмите Enter, чтобы пропустить): ")
url = input("Введите URL веб-страницы (или нажмите Enter, чтобы пропустить): ")

# Извлечение и анализ текста
if pdf_path:
    pdf_text = extract_text_from_pdf(pdf_path)
    print("Извлеченный текст из PDF:")
    print(pdf_text)
    analyze_text(pdf_text)

if docx_path:
    docx_text = extract_text_from_docx(docx_path)
    print("Извлеченный текст из DOCX:")
    print(docx_text)
    analyze_text(docx_text)

if doc_path:
    doc_text = extract_text_from_doc(doc_path)
    print("Извлеченный текст из DOC:")
    print(doc_text)
    analyze_text(doc_text)

if djvu_path:
    djvu_text = extract_text_from_djvu(djvu_path)
    print("Извлеченный текст из DJVU:")
    print(djvu_text)
    analyze_text(djvu_text)

if url:
    parse_html(url)

Выходные данные были обрезаны до нескольких последних строк (5000).
— (—) --> PUNCT [punct]
М. (м.) --> PROPN [ROOT]
, (,) --> PUNCT [punct]
1951 (1951) --> NUM [conj]
. (.) --> PUNCT [punct]
20 (20) --> NUM [ROOT]
. (.) --> PUNCT [punct]
Эльконин (эльконин) --> PROPN [nsubj]
Д. (д.) --> PROPN [appos]
Б. (б.) --> PROPN [flat:name]
Психологические (психологический) --> ADJ [amod]
основы (основа) --> NOUN [nsubj]
овладения (овладение) --> NOUN [nmod]
приемами (приём) --> NOUN [nmod]
стрельбы.//Военно (стрельбы.//военно) --> PROPN [amod]
- (-) --> ADJ [amod]
педагогический (педагогический) --> ADJ [amod]
сборник (сборник) --> NOUN [ROOT]
. (.) --> PUNCT [punct]
— (—) --> PUNCT [punct]
1952 (1952) --> NUM [ROOT]
, (,) --> PUNCT [punct]
№ (№) --> SYM [nummod:entity]
11 (11) --> NUM [appos]
. (.) --> PUNCT [punct]
21 (21) --> NUM [ROOT]
. (.) --> PUNCT [punct]
Эльконин (эльконин) --> PROPN [ROOT]
Д. (д.) --> PROPN [appos]
Б. (б.) --> PROPN [flat:name]
Особенности (особенность) --> NOUN [appo